In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time

from datetime import date, datetime
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

sns.set()

## Load data

In [ ]:
PATH = "/content/drive/MyDrive/DS300.N11/Final_Project/Dataset/"

In [ ]:
df_user = pd.read_csv(PATH+'User_data.csv', na_values='?')
df_user

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


In [ ]:
/content/drive/MyDrive/DS300.N11/Final_Project/Dataset/Dataset_VNW

In [ ]:
df_vnw = pd.read_csv(PATH+'Dataset_VNW', na_values='?')
df_vnw

,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://www.vietnamworks.com/senior-expert-eco...,"Senior Expert, Economics and Financial Markets...",Techcombank,Ngân hàng,"Head Office: 6 Quang Trung Street, Hoan Kiem D...",Job Purpose. The job holder responsible for:. ...,"Degree: University degree in Finance, Economic...",13th month salary + performance bonus. Advance...,"$ 1,000-2,500 /tháng",Hết hạn trong 6 ngày,Trưởng phòng,"Banking, Financial Analysis, Macro Analysis, F...",Ngân Hàng & Dịch Vụ Tài Chính > Phân Tích & Bá...,10,"Số 6 Quang Trung, Hoàn Kiếm, HN"
1,https://www.vietnamworks.com/audit-senior-ha-n...,Audit Senior ( Ha Noi & HCM),"Crowe Vietnam Co., Ltd",Kế toán/Kiểm toán,"HAN Office: Room 1205, 12th Floor, IPH Tower, ...",1. The primary responsibility is to assist aud...,University degree in Auditing/Accounting/ Fina...,Work for an international brand that frequentl...,Thương lượng,Hết hạn trong 2 ngày,Nhân viên,"Audit, Analytical Skills, Kiểm Toán Nội Bộ, En...",Kế Toán/Kiểm Toán > Kiểm Toán,3,"Hồ Chí Minh, Việt Nam"
2,https://www.vietnamworks.com/qa-engineer-ky-su...,"QA Engineer (Kỹ Sư Chất Lượng, Tiếng Anh Thành...",Công Ty TNHH LG Innotek Việt Nam Hải Phòng,Điện/Điện tử,"Lô I, 4, Khu công nghiệp Tràng Duệ, Xã Hồng Ph...",Process Quality Assurance. The Process Quality...,Well basis quality concept. English communicat...,Công ty cung cấp ký túc xá (nếu cần thiết). Xe...,"Tới $ 1,000 /tháng",Hết hạn trong 19 ngày,Nhân viên,"Process Quality Assurance, Excel, Test Plannin...",Sản Xuất > Đảm Bảo Chất Lượng/Kiểm Soát Chất L...,1,"Hai Phong, Vietnam"
3,https://www.vietnamworks.com/truong-nhom-kinh-...,Trưởng Nhóm Kinh Doanh,AIA Exchange HCM,Bảo hiểm,"Vincom Center, 45 Lý Tự Trọng, Phường Bến Nghé...",Lên kế hoạch tuyển dụng và phát triển đội ngũ ...,Độ tuổi: 27 40. Có kinh nghiệm ít nhất 1 năm l...,Thu nhập: Ít nhất từ 25 40 triệu mỗi tháng. Tr...,25tr-40tr ₫/tháng,Hết hạn trong 27 ngày,Nhân viên,"Quản Trị Đội Ngũ, Kỹ năng bán hàng, Quản Lý Tà...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,1,"Tầng 15, Vincom Đồng Khởi, 45A Lý Tự Trọng, Q1..."
4,https://www.vietnamworks.com/truong-phong-kinh...,Trưởng Phòng Kinh Doanh Kênh Lẻ (Chi nhánh Miề...,Công Ty Cổ Phần Anh Khuê Watch,Bán lẻ/Bán sỉ,"20 đường 3/2 Phường 12, quận 10, TP.HCM",Xây dựng và triển khai kế hoạch kinh doanh địn...,Giới tính: Nam/ Nữ (1990 – 1980). Tốt nghiệp Đ...,Thưởng theo hiệu quả công việc. Đầy đủ các chế...,Thương lượng,Hết hạn trong 23 ngày,Trưởng phòng,"Quản Lý Kinh Doanh Chuỗi, Kinh Doanh, Lập Kế H...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,5,"Số 35 phố Cự Lộc, Km 6 Nguyễn Trãi, Phường Thư..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6174,https://www.vietnamworks.com/pqa-manager-staff...,PQA Manager/ Staff,Navigos Search's Client,Điện/Điện tử,Hanoi,Làm việc với các bộ phận liên quan để tiến hàn...,1. Tốt nghiệp trường đại học chuyên ngành tươn...,"13th monthly salary, Performance Bonus. Health...","$ 1,100-1,800 /tháng",Hết hạn trong 4 ngày,Nhân viên,"Quality Assurance, Product Quality Assurance, ...",Sản Xuất > Đảm Bảo Chất Lượng/Kiểm Soát Chất L...,Không yêu cầu,Ha Nam
6175,https://www.vietnamworks.com/export-sales-exec...,Export Sales Executive,Navigos Search's Client,Nông nghiệp/Lâm nghiệp/Nuôi trồng thủy sản,Hanoi,"Navigos Search's client, FDI company in Bac Ni...","Male, 2530 years old. Have experience in expor...","13th monthly salary, Performance Bonus. Health...","$ 700-1,000 /tháng",Hết hạn trong 4 ngày,Nhân viên,"Export, Sales, English",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,Không yêu cầu,Thành phố Hà Nội
6176,https://www.vietnamworks.com/hot-job-demand-pl...,Hot Job! Demand Planner (SAP S/4 Hana),Navigos Search's Client,Hoá chất/Hoá sinh,Hanoi / HCMC,NaN,NaN,"13th monthly salary, Performance Bonus. Health...","$ 1,200-1,800 /tháng",Hết hạn trong 4 ngày,Nhân viên,

In [ ]:
df_jobsgo = pd.read_csv(PATH+'Dataset_Jobsgo', na_values='?')
df_jobsgo

,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://jobsgo.vn/viec-lam/nhan-vien-tu-van-tr...,"Nhân Viên Tư Vấn Trả Góp - Hà Nội, Đồng Nai, B...",Công ty Tài Chính TNHH MTV Home Credit Việt Nam,Tài chính,"20 Nguyễn Đăng Giai, P.Thảo Điền, Tp.Thủ Đức, ...",Tìm kiếm khách hàng thông qua data công công t...,"Nam/Nữ, từ 18 – 33 tuổi. Tốt nghiệp THPT trở l...",Thu nhập từ8.000.000 15.000.000đồng/tháng (hưở...,8 - 15 triệu VNĐ,Hết hạn trong 20 ngày nữa,Nhân Viên/Chuyên Viên,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Bán Lẻ","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng, Bán ...",Không yêu cầu,Đồng Nai
1,https://jobsgo.vn/viec-lam/nhan-vien-telesales...,Nhân Viên Telesales (Cho Vay Tiền Mặt),Công Ty Tài chính TNHH HD SAISON,Kinh doanh,"Lầu 8,9,10 Tòa nhà Gilimex, số 24C đường Phan ...",Đại diện HD SAISON trao đổi với khách hàng tro...,1. Trình độ học vấn:Tốt nghiệp Trung học phổ t...,Thử việc 100% lương;Ký hợp đồng lao động chính...,8 - 15 triệu VNĐ,Hết hạn trong 25 ngày nữa,Nhân Viên/Chuyên Viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Tài Chính/Ngân Hàng, Kinh Doanh/Bán Hàng, Bán ...",Không yêu cầu,"Tầng 8, 24C Phan Đăng Lưu, P6, Bình Thạnh, HCM"
2,https://jobsgo.vn/viec-lam/nhan-vien-kinh-doan...,Nhân Viên Kinh Doanh/ Sales/ Chăm Sóc Khách Hà...,Công Ty TNHH Tư Vấn & Đào Tạo Đại Dương,Giáo dục/ Đào tạo/ Tin học,"46A Đường 385, Phường Tăng Nhơn Phú A, Thành p...","1. Tìm kiếm, xây dựng mối quan hệ và phát triể...",Tốt nghiệp Cao Đẳng trở lên.Tuổi không quá 30....,Lương cứng từ9.000.000 – 12.000.000VNĐ/tháng+ ...,9 - 12 triệu VNĐ,Hết hạn trong 22 ngày nữa,Nhân Viên/Chuyên Viên,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng","Tài Chính/Ngân Hàng, Giáo Dục/Đào Tạo, Kinh Do...",Trên 1 năm,"25 Đường Số 23, Linh Chiểu, Thủ Đức, Hồ Chí Minh"
3,https://jobsgo.vn/viec-lam/ke-toan-tong-hop-di...,Kế Toán Tổng Hợp - Đi Làm Ngay (Chỉ Chấp Nhận ...,Công Ty TNHH TravelCraft,Kinh doanh,"L18-11-13, Tầng 18, Tòa nhà Vincom Center Đồng...",Phụ trách thực hiện các công việc kế toán của ...,"Nam/Nữ, 25 40 tuổi;Tốt nghiệp Đại học chuyên n...",Lương: 25 Tr 30 Tr (Thỏa thuận theo năng lực ứ...,25 - 30 triệu VNĐ,Hết hạn trong 22 ngày nữa,Nhân Viên/Chuyên Viên,"Kế Toán Tổng Hợp, Kế Toán Quản Trị, Misa","Tài Chính/Ngân Hàng, Du Lịch, Kế Toán/Kiểm Toán",Trên 5 năm,"64 Bà Huyện Thanh Quan, Võ Thị Sáu, Quận 3, Hồ..."
4,https://jobsgo.vn/viec-lam/ke-toan-ban-hang-19...,Kế Toán Bán Hàng,Công Ty TNHH Thương Mại Và Xuất Nhập Khẩu Bình...,Sản xuất/Vận hành sản xuất,"Số 248, đường Nguyễn Văn Linh, tổ 17, Phường T...",Thực hiện các nghiệp vụ kế toán liên quan đến ...,Nữ tuổi từ 22 – 33KHÔNG CÓ KINH NGHIỆM CÓ THỂ ...,Thu Nhập: 8 10 triệu ( có thể deal theo năng l...,8 - 10 triệu VNĐ,Hết hạn trong 28 ngày nữa,Nhân Viên/Chuyên Viên,"Kế Toán Tổng Hợp, Kế Toán Bán Hàng, Tư Vấn Bán...","Kế Toán/Kiểm Toán, Kinh Doanh/Bán Hàng, Bán Sỉ...",Không yêu cầu,"244 Nguyễn Văn Linh, Cổ Bi, Gia Lâm, Hà Nội"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,https://jobsgo.vn/viec-lam/nhan-vien-sale-1920...,Nhân Viên Sale,Công Ty TNHH Nhôm Tiến Đạt,Sản xuất/Vận hành sản xuất,"135A/3 KP.Bình Đường 3, Bình An, Dĩ An, Bình D...",Đi khắp 13 tỉnh ( có hỗ trợ xe hơi đi) tiếp th...,Liên hệ [protected info]( a duc) để xếp lịch p...,"Môi trường làm việc dân chủ, hiện đại, chuyên ...",10 - 12 triệu VNĐ,Hết hạn trong 8 ngày nữa,Nhân viên/Chuyên viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng",Không yêu cầu,Hậu Giang
9512,https://jobsgo.vn/viec-lam/fresher-mobile-flut...,Fresher Mobile (Flutter),Công Ty Cổ Phần Voyager,Công nghệ thông tin,"Floor 07, HOSE Building, Street 03, Quang Trun...","Tham gia học hỏi, xây dựng, phát triển các ứng...",Tốt nghiệp Đại học hoặc Cao đẳng trở lên các c...,Sẽ trở thành thành viên của một Đại Gia Đình đ...,Thỏa thuận,Hết hạn trong 8 ngày nữa,Mới tốt nghiệp/Thực tậ

In [ ]:
df_job = pd.concat([df_vnw, df_jobsgo], axis=0, ignore_index=True)

In [ ]:
df_job

,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://www.vietnamworks.com/senior-expert-eco...,"Senior Expert, Economics and Financial Markets...",Techcombank,Ngân hàng,"Head Office: 6 Quang Trung Street, Hoan Kiem D...",Job Purpose. The job holder responsible for:. ...,"Degree: University degree in Finance, Economic...",13th month salary + performance bonus. Advance...,"$ 1,000-2,500 /tháng",Hết hạn trong 6 ngày,Trưởng phòng,"Banking, Financial Analysis, Macro Analysis, F...",Ngân Hàng & Dịch Vụ Tài Chính > Phân Tích & Bá...,10,"Số 6 Quang Trung, Hoàn Kiếm, HN"
1,https://www.vietnamworks.com/audit-senior-ha-n...,Audit Senior ( Ha Noi & HCM),"Crowe Vietnam Co., Ltd",Kế toán/Kiểm toán,"HAN Office: Room 1205, 12th Floor, IPH Tower, ...",1. The primary responsibility is to assist aud...,University degree in Auditing/Accounting/ Fina...,Work for an international brand that frequentl...,Thương lượng,Hết hạn trong 2 ngày,Nhân viên,"Audit, Analytical Skills, Kiểm Toán Nội Bộ, En...",Kế Toán/Kiểm Toán > Kiểm Toán,3,"Hồ Chí Minh, Việt Nam"
2,https://www.vietnamworks.com/qa-engineer-ky-su...,"QA Engineer (Kỹ Sư Chất Lượng, Tiếng Anh Thành...",Công Ty TNHH LG Innotek Việt Nam Hải Phòng,Điện/Điện tử,"Lô I, 4, Khu công nghiệp Tràng Duệ, Xã Hồng Ph...",Process Quality Assurance. The Process Quality...,Well basis quality concept. English communicat...,Công ty cung cấp ký túc xá (nếu cần thiết). Xe...,"Tới $ 1,000 /tháng",Hết hạn trong 19 ngày,Nhân viên,"Process Quality Assurance, Excel, Test Plannin...",Sản Xuất > Đảm Bảo Chất Lượng/Kiểm Soát Chất L...,1,"Hai Phong, Vietnam"
3,https://www.vietnamworks.com/truong-nhom-kinh-...,Trưởng Nhóm Kinh Doanh,AIA Exchange HCM,Bảo hiểm,"Vincom Center, 45 Lý Tự Trọng, Phường Bến Nghé...",Lên kế hoạch tuyển dụng và phát triển đội ngũ ...,Độ tuổi: 27 40. Có kinh nghiệm ít nhất 1 năm l...,Thu nhập: Ít nhất từ 25 40 triệu mỗi tháng. Tr...,25tr-40tr ₫/tháng,Hết hạn trong 27 ngày,Nhân viên,"Quản Trị Đội Ngũ, Kỹ năng bán hàng, Quản Lý Tà...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,1,"Tầng 15, Vincom Đồng Khởi, 45A Lý Tự Trọng, Q1..."
4,https://www.vietnamworks.com/truong-phong-kinh...,Trưởng Phòng Kinh Doanh Kênh Lẻ (Chi nhánh Miề...,Công Ty Cổ Phần Anh Khuê Watch,Bán lẻ/Bán sỉ,"20 đường 3/2 Phường 12, quận 10, TP.HCM",Xây dựng và triển khai kế hoạch kinh doanh địn...,Giới tính: Nam/ Nữ (1990 – 1980). Tốt nghiệp Đ...,Thưởng theo hiệu quả công việc. Đầy đủ các chế...,Thương lượng,Hết hạn trong 23 ngày,Trưởng phòng,"Quản Lý Kinh Doanh Chuỗi, Kinh Doanh, Lập Kế H...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,5,"Số 35 phố Cự Lộc, Km 6 Nguyễn Trãi, Phường Thư..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15690,https://jobsgo.vn/viec-lam/nhan-vien-sale-1920...,Nhân Viên Sale,Công Ty TNHH Nhôm Tiến Đạt,Sản xuất/Vận hành sản xuất,"135A/3 KP.Bình Đường 3, Bình An, Dĩ An, Bình D...",Đi khắp 13 tỉnh ( có hỗ trợ xe hơi đi) tiếp th...,Liên hệ [protected info]( a duc) để xếp lịch p...,"Môi trường làm việc dân chủ, hiện đại, chuyên ...",10 - 12 triệu VNĐ,Hết hạn trong 8 ngày nữa,Nhân viên/Chuyên viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng",Không yêu cầu,Hậu Giang
15691,https://jobsgo.vn/viec-lam/fresher-mobile-flut...,Fresher Mobile (Flutter),Công Ty Cổ Phần Voyager,Công nghệ thông tin,"Floor 07, HOSE Building, Street 03, Quang Trun...","Tham gia học hỏi, xây dựng, phát triển các ứng...",Tốt nghiệp Đại học hoặc Cao đẳng trở lên các c...,Sẽ trở thành thành viên của một Đại Gia Đình đ...,Thỏa thuận,Hết hạn trong 8 ngày nữa,Mới tốt nghiệp/Thực tập sinh,"iOS, Android, Triển Khai Phần Mềm",IT Phần Mềm,Không yêu cầu,"Công viên phần mềm Quang Trung,Hồ Chí Minh"
15692,https://jobsgo.vn/viec-lam/chuyen-vien-pha-che...,Chuyên Viên Pha Chế,Công TY TNHH Một Thành Viên Nlink Việt Nam,Mỹ phẩm,"76 Trần Đình Xu, P. Cô Gia

## User - Normalize data

In [ ]:
def normalize_text_user(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

In [ ]:
def normalize_salary_user(salary):
  salary = salary.replace('.', '')
  result = []
  if bool(re.search(r'\d', salary)):
    if 'triệu' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result

In [ ]:
def calculate_age_user(born):
  today = date.today()
  return today.year - born.year - ((today.month, today.day) < (born.month, born.day))


In [ ]:
def normalize_experience_user(experience):
  try:
    result = 'Chưa có kinh nghiệm'
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)):
      if 'Dưới' in experience:
        result = experience
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = experience

  return result


In [ ]:
def processing_user_data(data):
  # Xóa cột Current Place of Residence, Province, District
  if 'Province' in data.columns:
    del data['Current Place of Residence']
    del data['Province']
    del data['District']
  # Xử lý missing values và dữ liệu trùng lặp
  data.dropna(subset=['Target', 'Skills'], inplace=True)
  data.drop_duplicates()
  data.reset_index(inplace=True)
  del data['index']
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):
    try:
      # Xử lý cột User Name
      try:
        data.loc[i,'User Name'] = normalize_text_user(data.loc[i,'User Name'])
      except:
        data.loc[i,'User Name'] = 'Không xác định'
      # Xử lý cột Industry
      try:
        lst_industry = data.loc[i,'Industry'].split(',')
        if 'span' in lst_industry[0]:
          k = []
          for l in lst_industry:
            pattern = re.compile(r'\>(.+?)\<')
            k.extend(pattern.findall(l))
          data.loc[i,'Industry'] = ', '.join(k)
      except:
        data.loc[i,'Industry'] = 'Không xác định'
      # Xử lý cột Desired Job
      try:
        data.loc[i,'Desired Job'] = normalize_text_user(data.loc[i,'Desired Job'])
      except:
        data.loc[i,'Desired Job'] = 'Không xác định'
      # Xử lý cột Workplace Desired
      try:
        lst_workplace_desired = data.loc[i,'Workplace Desired'].split(',')
        if '[' in lst_workplace_desired[0]:
          k = []
          for l in lst_workplace_desired:
            pattern = re.compile(r"\'(.+?)\'")
            k.extend(pattern.findall(l))
          data.loc[i,'Workplace Desired'] = ', '.join(k)
      except:
        data.loc[i,'Workplace Desired'] = 'Không xác định'
      # Xử lý cột Desired Salary
      salary = min(normalize_salary_user(data.loc[i,'Desired Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Desired Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Desired Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Desired Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Desired Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Desired Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Desired Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Desired Salary'] = salary
      # Xử lý cột Gender
      data['Gender'].fillna(value='Khác', inplace = True)
      # Xử lý cột Marriage
      data['Marriage'].fillna(value='Khác', inplace = True)
      # Xử lý cột Date of Birth -> chuyển thành cột Age
      if 'Date of Birth' in data.columns:
        try:
          DATE = datetime.strptime(data.loc[i,'Date of Birth'],"%d/%m/%Y").date()
          data.loc[i,'Date of Birth'] = calculate_age_user(DATE)
        except:
          data.loc[i,'Date of Birth'] = data.loc[i,'Date of Birth']
      # Xử lý cột Target
      data.loc[i,'Target'] = normalize_text_user(data.loc[i,'Target'])
      # Xử lý cột Skills
      data.loc[i,'Skills'] = normalize_text_user(data.loc[i,'Skills'])
      # Xử lý cột Target
      data.loc[i,'Degree'] = normalize_text_user(data.loc[i,'Degree'])
      # Xử lý cột Work Experience
      data.loc[i,'Work Experience'] = normalize_experience_user(data.loc[i,'Work Experience'])
    except:
      continue
  # Đổi tên cột Date of Birth -> Age
  data.rename(columns={'Date of Birth': 'Age'}, inplace=True)
  return data

In [ ]:
%%time
processing_user_data(df_user)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
<ipython-input-53-a5d8bc892e64>:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Gender'].fillna(value='Khác', inplace = True)
<ipython-input-53-a5d8bc892e64>:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

CPU times: user 26.8 s, sys: 745 ms, total: 27.5 s
Wall time: 33.4 s


,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3983 entries, 0 to 3982
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   URL User           3983 non-null   object
 1   UserID             3983 non-null   int64 
 2   User Name          3983 non-null   object
 3   Industry           3983 non-null   object
 4   Desired Job        3983 non-null   object
 5   Workplace Desired  3983 non-null   object
 6   Desired Salary     3983 non-null   object
 7   Gender             3983 non-null   object
 8   Marriage           3983 non-null   object
 9   Age                3983 non-null   int64 
 10  Target             3983 non-null   object
 11  Skills             3983 non-null   object
 12  Degree             3983 non-null   object
 13  Work Experience    3983 non-null   object
dtypes: int64(2), object(12)
memory usage: 435.8+ KB


## Job data

In [ ]:
def normalize_text_job(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

In [ ]:
def normalize_address(job_address):
  if '[' in job_address:
    k = []
    job_address = re.sub(" '',", '', job_address)
    job_address = re.sub("'Thu gọn'", '', job_address)
    pattern = re.compile(r"\'(.+?)\'")
    k.extend(pattern.findall(job_address))
    job_address = ', '.join(k)

  return job_address


In [ ]:

def normalize_salary_job(salary):
  salary = salary.replace('.', '')
  salary = salary.replace(',', '')
  salary = salary.replace('Thoả thuận', 'Thỏa thuận')
  result = []
  if bool(re.search(r'\d', salary)):
    if 'triệu' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result


In [ ]:
def normalize_experience_job(experience):
  try:
    result = 'Không yêu cầu kinh nghiệm'
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)):
      if 'Dưới' in experience or number < 1:
        result = 'Dưới 1 năm'
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = 'Không yêu cầu kinh nghiệm'

  return result

In [ ]:
def normalize_industry(industry):
  if '[' in industry:
    k = []
    pattern = re.compile(r"\'(.+?)\'")
    k.extend(pattern.findall(industry))
    industry = ', '.join(k)

  return industry

In [ ]:
def processing_job_data(data):
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):
    try:
      # Xử lý cột Job Title
      data.loc[i,'Job Title'] = normalize_text_job(data.loc[i,'Job Title'])

      # Xử lý cột Name Company
      data.loc[i,'Company Name'] = normalize_text_job(data.loc[i,'Company Name'])

      # Xử lý cột Company Address
      data.loc[i,'Company Address'] = normalize_address(data.loc[i,'Company Address'])

      # Xử lý cột Job Description
      data.loc[i,'Job Description'] = re.sub('Mô tả công việc', '', data.loc[i,'Job Description'])
      data.loc[i,'Job Description'] = normalize_text_job(data.loc[i,'Job Description'])

      # Xử lý cột Job Requirements
      data.loc[i,'Job Requirements'] = re.sub('Yêu cầu công việc', '', data.loc[i,'Job Requirements'])
      data.loc[i,'Job Requirements'] = normalize_text_job(data.loc[i,'Job Requirements'])

      # Xử lý cột Benefits
      data.loc[i,'Benefits'] = re.sub('Quyền lợi được hưởng', '', data.loc[i,'Benefits'])
      data.loc[i,'Benefits'] = normalize_text_job(data.loc[i,'Benefits'])

      # Xử lý cột Job Address
      data.loc[i,'Job Address'] = normalize_address(data.loc[i,'Job Address'])

      # Xử lý cột Job Type
      data.loc[i,'Job Field'] = normalize_text_job(data.loc[i,'Job Field'])

      # Xử lý cột Job Skill
      data.loc[i,'Job Skill'] = normalize_text_job(data.loc[i,'Job Skill'])

      # Xử lý cột Years of Experience
      data.loc[i,'Years of Experience'] = normalize_experience_job(data.loc[i,'Years of Experience'])
      # Xử lý cột Salary
      salary = min(normalize_salary_job(data.loc[i,'Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Salary'] = salary
      # Xử lý cột Expire Date
      data.loc[i,'Expire date'] = normalize_text_job(data.loc[i,'Expire date'])

      # Xử lý cột Employee Rank
      data.loc[i,'Employee Rank'] = normalize_text_job(data.loc[i,'Employee Rank'])
      # Xử lý cột Industry
      try:
        data.loc[i,'Industry'] = normalize_industry(data.loc[i,'Industry'])
      except:
        data.loc[i,'Industry'] = 'Không xác định'
    except:
      print("Error")

  return data


In [ ]:

%%time
processing_job_data(df_job)

  4%|▍         | 632/15509 [00:01<00:31, 474.37it/s]

Error


  5%|▍         | 729/15509 [00:01<00:32, 457.21it/s]

Error
Error


  8%|▊         | 1201/15509 [00:02<00:46, 309.64it/s]

...

100%|█████████▉| 15438/15509 [00:39<00:00, 479.74it/s]

Error
Error
Error
Error
Error
Error
Error
Error
Error


100%|██████████| 15509/15509 [00:40<00:00, 386.74it/s]


CPU times: user 38.3 s, sys: 295 ms, total: 38.6 s
Wall time: 40.1 s


,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://www.vietnamworks.com/senior-expert-eco...,"Senior Expert, Economics and Financial Markets...",Techcombank,Ngân hàng,"Head Office: 6 Quang Trung Street, Hoan Kiem D...",Job Purpose. The job holder responsible for:. ...,"Degree: University degree in Finance, Economic...",13th month salary + performance bonus. Advance...,"Trên 40,000,000",Hết hạn trong 6 ngày,Trưởng phòng,"Banking, Financial Analysis, Macro Analysis, F...",Ngân Hàng & Dịch Vụ Tài Chính > Phân Tích & Bá...,Trên 10 năm,"Số 6 Quang Trung, Hoàn Kiếm, HN"
1,https://www.vietnamworks.com/audit-senior-ha-n...,Audit Senior ( Ha Noi & HCM),"Crowe Vietnam Co., Ltd",Kế toán/Kiểm toán,"HAN Office: Room 1205, 12th Floor, IPH Tower, ...",1. The primary responsibility is to assist aud...,University degree in Auditing/Accounting/ Fina...,Work for an international brand that frequentl...,Thương lượng,Hết hạn trong 2 ngày,Nhân viên,"Audit, Analytical Skills, Kiểm Toán Nội Bộ, En...",Kế Toán/Kiểm Toán > Kiểm Toán,3-5 năm,"Hồ Chí Minh, Việt Nam"
2,https://www.vietnamworks.com/qa-engineer-ky-su...,"QA Engineer (Kỹ Sư Chất Lượng, Tiếng Anh Thành...",Công Ty TNHH LG Innotek Việt Nam Hải Phòng,Điện/Điện tử,"Lô I, 4, Khu công nghiệp Tràng Duệ, Xã Hồng Ph...",Process Quality Assurance. The Process Quality...,Well basis quality concept. English communicat...,Công ty cung cấp ký túc xá (nếu cần thiết). Xe...,"Trên 40,000,000",Hết hạn trong 19 ngày,Nhân viên,"Process Quality Assurance, Excel, Test Plannin...",Sản Xuất > Đảm Bảo Chất Lượng/Kiểm Soát Chất L...,1-3 năm,"Hai Phong, Vietnam"
3,https://www.vietnamworks.com/truong-nhom-kinh-...,Trưởng Nhóm Kinh Doanh,AIA Exchange HCM,Bảo hiểm,"Vincom Center, 45 Lý Tự Trọng, Phường Bến Nghé...",Lên kế hoạch tuyển dụng và phát triển đội ngũ ...,Độ tuổi: 27 40. Có kinh nghiệm ít nhất 1 năm l...,Thu nhập: Ít nhất từ 25 40 triệu mỗi tháng. Tr...,"20,000,000 - 40,000,000",Hết hạn trong 27 ngày,Nhân viên,"Quản Trị Đội Ngũ, Kỹ năng bán hàng, Quản Lý Tà...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,1-3 năm,"Tầng 15, Vincom Đồng Khởi, 45A Lý Tự Trọng, Q1..."
4,https://www.vietnamworks.com/truong-phong-kinh...,Trưởng Phòng Kinh Doanh Kênh Lẻ (Chi nhánh Miề...,Công Ty Cổ Phần Anh Khuê Watch,Bán lẻ/Bán sỉ,"20 đường 3/2 Phường 12, quận 10, TP.HCM",Xây dựng và triển khai kế hoạch kinh doanh địn...,Giới tính: Nam/ Nữ (1990 – 1980). Tốt nghiệp Đ...,Thưởng theo hiệu quả công việc. Đầy đủ các chế...,Thương lượng,Hết hạn trong 23 ngày,Trưởng phòng,"Quản Lý Kinh Doanh Chuỗi, Kinh Doanh, Lập Kế H...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,5-10 năm,"Số 35 phố Cự Lộc, Km 6 Nguyễn Trãi, Phường Thư..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15690,https://jobsgo.vn/viec-lam/nhan-vien-sale-1920...,Nhân Viên Sale,Công Ty TNHH Nhôm Tiến Đạt,Sản xuất/Vận hành sản xuất,"135A/3 KP.Bình Đường 3, Bình An, Dĩ An, Bình D...",Đi khắp 13 tỉnh ( có hỗ trợ xe hơi đi) tiếp th...,Liên hệ [protected info]( a duc) để xếp lịch p...,"Môi trường làm việc dân chủ, hiện đại, chuyên ...","10,000,000 - 15,000,000",Hết hạn trong 8 ngày nữa,Nhân viên/Chuyên viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng",Không yêu cầu kinh nghiệm,Hậu Giang
15691,https://jobsgo.vn/viec-lam/fresher-mobile-flut...,Fresher Mobile (Flutter),Công Ty Cổ Phần Voyager,Công nghệ thông tin,"Floor 07, HOSE Building, Street 03, Quang Trun...","Tham gia học hỏi, xây dựng, phát triển các ứng...",Tốt nghiệp Đại học hoặc Cao đẳng trở lên các c...,Sẽ trở thành thành viên của một Đại Gia Đình đ...,Thỏa thuận,Hết hạn trong 8 ngày nữa,Mới tốt nghiệp/Thực tập sinh,"iOS, Android, Triển Khai Phần Mềm",IT Phần Mềm,Không yêu cầu kinh nghiệm,"Công viên phần mềm Quang Trung,Hồ Chí Minh"
15692,https://jobsgo.vn/viec-lam/chuyen-vien-pha-che...,Chuyên Viên Pha Chế,Công TY TNHH

In [ ]:
df_job['Years of Experience'].value_counts()

,count
Years of Experience,
1-3 năm,6032
Không yêu cầu kinh nghiệm,5280
3-5 năm,1849
5-10 năm,1120
Dưới 1 năm,1103
Trên 10 năm,125


In [ ]:
salary_counts = df_job['Salary'].value_counts()
valid_salaries = salary_counts[salary_counts >= 100].index
df_job = df_job[df_job['Salary'].isin(valid_salaries)]

In [ ]:
Experiece_counts = df_job['Years of Experience'].value_counts()
valid_salaries = Experiece_counts[Experiece_counts >= 100].index
df_job = df_job[df_job['Years of Experience'].isin(valid_salaries)]

In [ ]:
df_user

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


In [ ]:
df_user.to_csv(PATH+'User_data_final.csv', index=False)
df_job.to_csv(PATH+'Job_data_final.csv', index=False)